In [1]:
import tensorflow as tf
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import os
import time

In [2]:
cdnet_root_path = '/home/Datasets/CDnet2014/dataset'
raw_bin_path = '/home/Datasets/SWCD_results/raw_bin'
SSA_bin_path = '/home/Datasets/SWCD_results/SSA_bin'

In [3]:
def calculate_base_scores(uint8_result,uint8_truth):
    pos_mask,neg_mask = calculate_double_mask(uint8_truth)
    
    result = np.float32(uint8_result)/255
    
    TP = np.sum(pos_mask*result)
    TN = np.sum(neg_mask*(1.0-result))
    FP = np.sum(neg_mask*result)
    FN = np.sum(pos_mask*(1.0-result))
    
    return [TP,TN,FP,FN]

In [4]:
def calculate_synthesis_scores(base_scores):
    
    TP = base_scores[0]
    TN = base_scores[1]
    FP = base_scores[2]
    FN = base_scores[3]

    Recall = np.maximum(1e-3,TP) / np.maximum(1e-3,TP + FN)
    Specificity = np.maximum(1e-3,TN) / np.maximum(1e-3,TN + FP)
    PWC = 100.0 * np.maximum(1e-3,FN + FP) / np.maximum(1e-3,TP + FN + FP + TN)
    Precision = np.maximum(1e-3,TP) / np.maximum(1e-3,TP + FP)
    F_Measure = (2 * Precision * Recall) / (Precision + Recall)
    
    return [Recall,Specificity,PWC,Precision,F_Measure]

In [5]:
def calculate_double_mask(ground_truth):
    """Calculate the weight for calculating the loss of the model"""
   
    positive_mask = np.zeros_like(np.float32(ground_truth))
    negative_mask = np.zeros_like(np.float32(ground_truth))

    index_object = np.where(ground_truth==255)
    index_background = np.where(ground_truth==0)
    
    positive_mask[index_object] = 1.0
    negative_mask[index_background] = 1.0
    
    return positive_mask, negative_mask

In [6]:
raw_bin_score_dict = {}
SSA_bin_score_dict = {}
for root, _, files in os.walk(cdnet_root_path):
    
    relative_path_hierarch_list = root.split('/')[len(cdnet_root_path.split('/')):]
    level_above_dataset_root_path = len(relative_path_hierarch_list)
    
    if len(relative_path_hierarch_list) == 2:
        
        print(relative_path_hierarch_list)
        temporalROI = np.loadtxt(root+'/temporalROI.txt').astype(np.int)
        print('{0:06d}-{1:06d}'.format(temporalROI[0],temporalROI[1]))

        raw_bin_base_scores = np.array([0.0,0.0,0.0,0.0])
        SSA_bin_base_scores = np.array([0.0,0.0,0.0,0.0])
        
        for file_index in range(temporalROI[0],temporalROI[1]):
            file_index_string = '{0:06d}'.format(file_index)
            
            truth_file_path = cdnet_root_path+'/'+relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]+'/groundtruth/gt'+file_index_string+'.png'
            raw_bin_file_path = raw_bin_path+'/'+relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]+'/bin'+file_index_string+'.png'
            SSA_bin_file_path = SSA_bin_path+'/'+relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]+'/bin'+file_index_string+'.png'
            
            #print(AFH_SS_post_bin_file_path)
            
            truth = cv2.imread(truth_file_path,0)
            raw_bin = cv2.imread(raw_bin_file_path,0)
            SSA_bin = cv2.imread(SSA_bin_file_path,0)
            
            raw_bin_base_scores += np.array(calculate_base_scores(raw_bin,truth))
            SSA_bin_base_scores += np.array(calculate_base_scores(SSA_bin,truth))
        
        raw_bin_synthesis_scores = calculate_synthesis_scores(raw_bin_base_scores)
        SSA_bin_synthesis_scores = calculate_synthesis_scores(SSA_bin_base_scores)
        
        print(raw_bin_base_scores)
        print(SSA_bin_base_scores)
        print(raw_bin_synthesis_scores)
        print(SSA_bin_synthesis_scores)
        
        raw_bin_score_dict[relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]] = np.hstack([raw_bin_base_scores,raw_bin_synthesis_scores])
        SSA_bin_score_dict[relative_path_hierarch_list[0]+'/'+relative_path_hierarch_list[1]] = np.hstack([SSA_bin_base_scores,SSA_bin_synthesis_scores])


['dynamicBackground', 'overpass']
001000-003000
[1.69113700e+06 1.45253113e+08 3.03115000e+05 2.86755000e+05]
[1.92852700e+06 1.45467943e+08 8.82850000e+04 4.93650000e+04]
[0.8550198898625405, 0.9979175401549977, 0.3998193773752133, 0.8480056682906674, 0.8514983344007669]
[0.9750416099564587, 0.9993934646341619, 0.09330045144811248, 0.9562254687100236, 0.9655418774457382]
['dynamicBackground', 'canoe']
000800-001189
[  980960. 28229443.   113478.    62409.]
[1.0056110e+06 2.8315115e+07 2.7806000e+04 3.7758000e+04]
[0.9401851118827568, 0.9959962489399028, 0.5985342144244816, 0.8963139072290984, 0.9177255009455952]
[0.9638114607583702, 0.9990189437426016, 0.22311084522748534, 0.9730931463291198, 0.9684300645324072]
['dynamicBackground', 'fountain01']
000400-001184
[5.5197000e+04 9.3151316e+07 1.2718000e+04 2.2306000e+04]
[5.5020000e+04 9.3157767e+07 6.2670000e+03 2.2483000e+04]
[0.712191786124408, 0.9998634880924113, 0.03756265836758997, 0.8127365088713834, 0.7591494863084349]
[0.7099080

[  668697. 13684537.   136377.    36248.]
[6.6565800e+05 1.3807359e+07 1.3555000e+04 3.9287000e+04]
[0.9485803857038493, 0.9901325628681287, 1.1883978771926673, 0.8306031495241432, 0.8856802464074957]
[0.9442694110888084, 0.9990192399721176, 0.36377883056692206, 0.9800430792696841, 0.9618237224363114]
['turbulence', 'turbulence3']
000800-002200
[3.02384900e+06 2.37806932e+08 2.85009000e+05 8.70737000e+05]
[2.8771390e+06 2.3805701e+08 3.4931000e+04 1.0174470e+06]
[0.776423732843491, 0.9988029456234304, 0.4776075818469018, 0.9138648440035807, 0.8395564677118335]
[0.7387534901013869, 0.9998532877683584, 0.4348911540847892, 0.9880047526330068, 0.8453898654493485]
['turbulence', 'turbulence1']
001200-004000
[1.13426400e+06 4.80490508e+08 2.94859000e+05 6.99905000e+05]
[1.00207600e+06 4.80748199e+08 3.71680000e+04 8.32093000e+05]
[0.618407573129848, 0.9993867138639434, 0.2061176404595441, 0.793678360784901, 0.6951654954567351]
[0.54633787835254, 0.9999226931546775, 0.1801130984469721, 0.9642

[1.02709360e+07 1.03800798e+09 9.30248000e+05 1.98558300e+06]
[1.02301390e+07 1.03844525e+09 4.92979000e+05 2.02638000e+06]
[0.837997803454635, 0.9991046166387805, 0.27738257032228847, 0.9169509223310679, 0.8756983580191121]
[0.8346692074642074, 0.999525497293162, 0.23966624779851448, 0.954026524747746, 0.8903655875852172]
['nightVideos', 'busyBoulvard']
000730-002760
[  622977. 31268596.   872721.   552993.]
[  607667. 31923002.   218315.   568303.]
[0.529755861118906, 0.9728473789670784, 4.2792019650339475, 0.41651255801639103, 0.46635809539209216]
[0.5167368215175557, 0.9932076523186651, 2.3609905572443517, 0.7356903661338867, 0.6070744952925945]
['nightVideos', 'tramStation']
000500-003000
[ 2264043. 88711714.  1188261.   277623.]
[ 2261970. 89290354.   609621.   279696.]
[0.8907712500383607, 0.9867824101174667, 1.5857399156295808, 0.6558063832153831, 0.7554402174185055]
[0.889955643267054, 0.9932188968906832, 0.9620307367758649, 0.7877061879633973, 0.8357149863751158]
['nightVideo

In [7]:
np.save('raw_bin_score_dict',raw_bin_score_dict)
np.save('SSA_bin_score_dict',SSA_bin_score_dict)

In [8]:
raw_bin_score_dict=np.load('raw_bin_score_dict.npy').item()
SSA_bin_score_dict=np.load('SSA_bin_score_dict.npy').item()

In [9]:
raw_bin_level_1_category_list = [key.split('/')[0] for key in raw_bin_score_dict.keys()]
raw_bin_level_2_category_list = [key.split('/')[1] for key in raw_bin_score_dict.keys()]
raw_bin_level_2_F_list = [raw_bin_score_dict[key][-1] for key in raw_bin_score_dict.keys()]
raw_bin_level_1_category_set = set(raw_bin_level_1_category_list)
raw_bin_level_1_category_F_score_dict = {it:[] for it in raw_bin_level_1_category_set}
for cat,F in zip(raw_bin_level_1_category_list,raw_bin_level_2_F_list):
    raw_bin_level_1_category_F_score_dict[cat].append(F)
raw_bin_level_1_category_average_F_score_dict = {key:np.mean(raw_bin_level_1_category_F_score_dict[key]) for key in raw_bin_level_1_category_F_score_dict}
raw_bin_level_1_category_average_F_score_dict

{'shadow': 0.9220751128098303,
 'intermittentObjectMotion': 0.7121674589392524,
 'badWeather': 0.8490595532277975,
 'baseline': 0.9307842089253393,
 'dynamicBackground': 0.8664521596461725,
 'turbulence': 0.8392500419953469,
 'nightVideos': 0.5562439686318679,
 'PTZ': 0.4863243492270309,
 'lowFramerate': 0.7383304519470968,
 'cameraJitter': 0.7616138193904916,
 'thermal': 0.8581511586282549}

In [10]:
SSA_bin_level_1_category_list = [key.split('/')[0] for key in SSA_bin_score_dict.keys()]
SSA_bin_level_2_category_list = [key.split('/')[1] for key in SSA_bin_score_dict.keys()]
SSA_bin_level_2_F_list = [SSA_bin_score_dict[key][-1] for key in SSA_bin_score_dict.keys()]
SSA_bin_level_1_category_set = set(SSA_bin_level_1_category_list)
SSA_bin_level_1_category_F_score_dict = {it:[] for it in SSA_bin_level_1_category_set}
for cat,F in zip(SSA_bin_level_1_category_list,SSA_bin_level_2_F_list):
    SSA_bin_level_1_category_F_score_dict[cat].append(F)
SSA_bin_level_1_category_average_F_score_dict = {key:np.mean(SSA_bin_level_1_category_F_score_dict[key]) for key in SSA_bin_level_1_category_F_score_dict}
SSA_bin_level_1_category_average_F_score_dict

{'shadow': 0.9665060612761479,
 'intermittentObjectMotion': 0.8211191982899679,
 'badWeather': 0.9085305596200872,
 'baseline': 0.9533741938947402,
 'dynamicBackground': 0.9163863904814237,
 'turbulence': 0.8346355651401762,
 'nightVideos': 0.7168237545617355,
 'PTZ': 0.6849235069848494,
 'lowFramerate': 0.7993266747171935,
 'cameraJitter': 0.9064305004663011,
 'thermal': 0.9174419164761719}